In [ ]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error, r2_score
import time

# Load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

nn_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

total_vram_usage = 0
total_rmse = 0
total_r2 = 0
num_checks = 5

# NN model
for epoch in range(num_checks):
    start_time = time.time()

    with tf.device('/GPU:0'):
        nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=[])
        nn_model.fit(x_train, y_train, epochs=1, verbose=2)

        y_pred = nn_model.predict(x_test)

        rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_test, y_pred))))
        r2 = r2_score(y_test, y_pred)

        print(f"NN Model, Epoch {epoch + 1}/5 - RMSE: {rmse:.4f}, R-squared: {r2:.4f}")

    end_time = time.time()

    gpu_info = !nvidia-smi
    vram_usage_line = next((line for line in gpu_info if 'MiB /' in line), None)

    if vram_usage_line:
        vram_used_str = vram_usage_line.split()[8]
        vram_used = int(vram_used_str[:-3])

        print(f"VRAM Usage: {vram_used} MiB")

        total_vram_usage += vram_used
        total_rmse += rmse
        total_r2 += r2

        time.sleep(5)

average_vram_usage = total_vram_usage / num_checks
average_rmse = total_rmse / num_checks
average_r2 = total_r2 / num_checks
print(f"Average VRAM Usage: {average_vram_usage} MiB")
print(f"Average RMSE: {average_rmse:.4f}")
print(f"Average R-squared: {average_r2:.4f}")


11490434/11490434 [==============================] - 1s 0us/step
1 Physical GPUs, 1 Logical GPUs
1875/1875 - 12s - loss: 2.1100 - 12s/epoch - 7ms/step
313/313 [==============================] - 1s 2ms/step
NN Model, Epoch 1/5 - RMSE: 3.9130, R-squared: 0.8643
VRAM Usage: 2175 MiB
1875/1875 - 4s - loss: 1.0143 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 2ms/step
NN Model, Epoch 2/5 - RMSE: 3.9820, R-squared: 0.8880
VRAM Usage: 2175 MiB
1875/1875 - 4s - loss: 0.8315 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 2ms/step
NN Model, Epoch 3/5 - RMSE: 4.0207, R-squared: 0.8927
VRAM Usage: 2175 MiB
1875/1875 - 5s - loss: 0.7227 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
NN Model, Epoch 4/5 - RMSE: 4.0279, R-squared: 0.9066
VRAM Usage: 2175 MiB
1875/1875 - 4s - loss: 0.6557 - 4s/epoch - 2ms/step
313/313 [==============================] - 1s 2ms/step
NN Model, Epoch 5/5 - RMSE: 4.0831, R-squared: 0.9100
VRAM Usage: 